In [1]:
from fastbook import *
setup_book()

import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

In [2]:
to = load_pickle('..\\prepared_data\\to.pkl')

In [3]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

<font size = 6> Decision Trees </font>

In [5]:
dtree0 = DecisionTreeRegressor(min_samples_leaf=10)
dtree0.fit(xs,y)
#draw_tree(dtree0, xs, size=10, leaves_parallel=True, precision=2)

DecisionTreeRegressor(min_samples_leaf=10)

In [5]:
def r_mse(pred,y): # I will be using root mean square error to evaluate the perform of my model
    return round(math.sqrt(((pred-y)**2).mean()), 6)
def model_rmse(model, xs, y): 
    return r_mse(model.predict(xs), y) 

model_rmse(dtree0, xs, y)

In [7]:
# error on validation set:
model_rmse(dtree0, valid_xs, valid_y)

0.115187

In [8]:
# test various hyperparams
dtrees = []

for num in (5*x for x in range(1, 11)):
    tree = DecisionTreeRegressor(min_samples_leaf=num) # minimum number of data points in a leaf node
    tree.fit(xs,y)
    error = (model_rmse(tree, xs, y), model_rmse(tree, valid_xs, valid_y)) # train and valid errors, respectively
    dtrees.append((tree, error))

dtrees

[(DecisionTreeRegressor(min_samples_leaf=5), (0.069633, 0.113489)),
 (DecisionTreeRegressor(min_samples_leaf=10), (0.084575, 0.098594)),
 (DecisionTreeRegressor(min_samples_leaf=15), (0.094155, 0.115911)),
 (DecisionTreeRegressor(min_samples_leaf=20), (0.106207, 0.108794)),
 (DecisionTreeRegressor(min_samples_leaf=25), (0.109014, 0.108507)),
 (DecisionTreeRegressor(min_samples_leaf=30), (0.115178, 0.110678)),
 (DecisionTreeRegressor(min_samples_leaf=35), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=40), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=45), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=50), (0.119561, 0.122005))]

The second tree, which had min_samples_leaf = 10, has the lowest validation set error. I now also test various other hyper parameters

In [9]:
for num in (5*x for x in range(1, 11)):
    tree = DecisionTreeRegressor(min_samples_split=num) # minimum number of samples at which you can split the node
    tree.fit(xs,y)
    error = (model_rmse(tree, xs, y), model_rmse(tree, valid_xs, valid_y)) 
    dtrees.append((tree, error))

for num in range(1, 20):
    tree = DecisionTreeRegressor(max_depth=num) # max depth
    tree.fit(xs,y)
    error = (model_rmse(tree, xs, y), model_rmse(tree, valid_xs, valid_y)) 
    dtrees.append((tree, error))
    
for num in range(4, 80):
    tree = DecisionTreeRegressor(max_leaf_nodes=num) # max number of leaf nodes
    tree.fit(xs,y)
    error = (model_rmse(tree, xs, y), model_rmse(tree, valid_xs, valid_y)) 
    dtrees.append((tree, error))

dtrees

[(DecisionTreeRegressor(min_samples_leaf=5), (0.069633, 0.113489)),
 (DecisionTreeRegressor(min_samples_leaf=10), (0.084575, 0.098594)),
 (DecisionTreeRegressor(min_samples_leaf=15), (0.094155, 0.115911)),
 (DecisionTreeRegressor(min_samples_leaf=20), (0.106207, 0.108794)),
 (DecisionTreeRegressor(min_samples_leaf=25), (0.109014, 0.108507)),
 (DecisionTreeRegressor(min_samples_leaf=30), (0.115178, 0.110678)),
 (DecisionTreeRegressor(min_samples_leaf=35), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=40), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=45), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=50), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_split=5), (0.026998, 0.132626)),
 (DecisionTreeRegressor(min_samples_split=10), (0.055762, 0.097467)),
 (DecisionTreeRegressor(min_samples_split=15), (0.071154, 0.109353)),
 (DecisionTreeRegressor(min_samples_split=20), (0.079708, 0.109858)),
 (DecisionTreeRegressor(min_samp

In [10]:
best = min((tupl[1][1] for tupl in dtrees))
best 

0.081894

This validation set error would appear to come from the decision tree with min_samples_split=30, so we will choose it as our candidate decision tree for comparison with Random Forest and Neural Net models

In [12]:
dtree = DecisionTreeRegressor(min_samples_split=30)
dtree.fit(xs, y)
#draw_tree(dtree, xs, size=10, leaves_parallel=True, precision=3)

DecisionTreeRegressor(min_samples_split=30)

In [13]:
model_rmse(dtree, valid_xs, valid_y)

0.081894

Try nested hyperparameter tuning

In [14]:
for samples_leaf in (5*x for x in range(1, 11)):
    for samples_split in (5*x for x in range(1, 11)):
        for depth in range(1, 20):
            #for leaf_nodes in range(4, 80):
            tree = DecisionTreeRegressor(min_samples_leaf=samples_leaf, 
                                         min_samples_split=samples_split,
                                         max_depth=depth,
                                         #max_leaf_nodes=leaf_nodes
                                        )
            tree.fit(xs,y)
            error = (model_rmse(tree, xs, y), model_rmse(tree, valid_xs, valid_y)) 
            dtrees.append((tree, error))
                
dtrees

[(DecisionTreeRegressor(min_samples_leaf=5), (0.069633, 0.113489)),
 (DecisionTreeRegressor(min_samples_leaf=10), (0.084575, 0.098594)),
 (DecisionTreeRegressor(min_samples_leaf=15), (0.094155, 0.115911)),
 (DecisionTreeRegressor(min_samples_leaf=20), (0.106207, 0.108794)),
 (DecisionTreeRegressor(min_samples_leaf=25), (0.109014, 0.108507)),
 (DecisionTreeRegressor(min_samples_leaf=30), (0.115178, 0.110678)),
 (DecisionTreeRegressor(min_samples_leaf=35), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=40), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=45), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_leaf=50), (0.119561, 0.122005)),
 (DecisionTreeRegressor(min_samples_split=5), (0.026998, 0.132626)),
 (DecisionTreeRegressor(min_samples_split=10), (0.055762, 0.097467)),
 (DecisionTreeRegressor(min_samples_split=15), (0.071154, 0.109353)),
 (DecisionTreeRegressor(min_samples_split=20), (0.079708, 0.109858)),
 (DecisionTreeRegressor(min_samp

In [15]:
best = min((tupl[1][1] for tupl in dtrees))
best 

0.081894

So trying all the combinations of these hyperparameters didn't yield anything better than trying the hyperparameters separately

In [16]:
save_pickle('dtree_best.pkl',dtree)

<font size = 6> Random Forests</font>

In [17]:
def rf(xs, y, n_estimators=40, max_samples=184, # size of my training set
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, # means to use all available processors in parallel
                                 n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [18]:
rf0 = rf(xs, y, max_samples=184)

In [19]:
model_rmse(rf0, xs, y), model_rmse(rf0, valid_xs, valid_y)

(0.075594, 0.081931)

My first random forest, with no hyperparameter tuning, is comparable to the very best decision tree

In [20]:
forests = {}

for num in (5*x for x in range(1, 11)):
    forest = rf(xs, y, min_samples_leaf=num) # minimum number of samples at which you can split the node
    error = model_rmse(forest, valid_xs, valid_y)
    forests[error] = forest

from random import uniform
for num in (uniform(0,1) for i in range(21)):
    forest = rf(xs, y, max_features=num) # minimum number of samples at which you can split the node
    error = model_rmse(forest, valid_xs, valid_y)
    forests[error] = forest

In [21]:
forests

{0.085219: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=5,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.09455: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=10,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.099114: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=15,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.101754: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=20,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.101197: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=25,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.106159: RandomForestRegressor(max_features=0.5, max_samples=184, min_samples_leaf=30,
                       n_estimators=40, n_jobs=-1, oob_score=True),
 0.112601: RandomForestRegressor(max_features=0.5, max_sampl

In [22]:
best = (min(forests), forests[min(forests)])
best

(0.082807,
 RandomForestRegressor(max_features=0.6766994874229113, max_samples=184,
                       min_samples_leaf=5, n_estimators=40, n_jobs=-1,
                       oob_score=True))

The optimal maximum of what portion of the independent variables any individual tree in the forest can consider looks to be 0.805, giving a validation set error of 0.080. There also appears to be an increase in error as min_samples_leaf increases, so use min_samples_leaf =5 and max_features= 0.8 in the remaining combinations of hyperparameters I test.

In [25]:
for estimators in (5*x for x in range(1, 41)):
    for samples in (5*x for x in range(1,36)):
        forest = rf(xs, y, max_samples=samples, 
                    n_estimators=estimators, 
                    max_features=0.8, 
                    min_samples_leaf=5) 
        error = model_rmse(forest, valid_xs, valid_y)
        forests[error] = forest
        
forests

C:\Users\bbdil\OneDrive\Desktop\LeetcodeSummer\election_prediction_project_and_venv\venv\lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\bbdil\OneDrive\Desktop\LeetcodeSummer\election_prediction_project_and_venv\venv\lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\bbdil\OneDrive\Desktop\LeetcodeSummer\election_prediction_project_and_venv\venv\lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\bbdil\OneDrive\Desktop\LeetcodeSummer\election_prediction_project_and_venv\venv\lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some in

KeyboardInterrupt: 

In [26]:
best = (min(forests), forests[min(forests)])
best

(0.075351,
 RandomForestRegressor(max_features=0.8, max_samples=165, min_samples_leaf=5,
                       n_estimators=20, n_jobs=-1, oob_score=True))

The best-generalizing random forest, as measured by lowest error on validation set, appears to be one consisting of 20 trees which each are fit on at most 160 rows (max_samples)

In [27]:
rf_best = rf(xs, y, n_estimators=20, max_samples=165, max_features=0.8, min_samples_leaf=5)

In [28]:
model_rmse(rf_best, valid_xs, valid_y)

0.088113

In [29]:
# The level of stochasticity is high and thus creating a new random forest with the same arguments won't give the 
# same random forest, so we need to get the exact model that gave such a low error
rf_best = best[1]

In [30]:
model_rmse(rf_best, valid_xs, valid_y)

0.075351

In [31]:
save_pickle('rf_best.pkl',rf_best)

<font size=6> Neural Networks</font>

In [6]:
df_nn = pd.read_csv('..\\prepared_data\\dataset_7-24.csv', low_memory=False)

cond = df_nn.year<=2010
import numpy as np
train_idx = np.where(cond)[0] # if condition only param passed, returns indices of dataframe where conditions true
valid_idx = np.where(~cond)[0] # why the [0] ?
splits = (list(train_idx),list(valid_idx))

In [7]:
df_nn.drop(columns=['year', 'Dem_candidate', 'Repub_candidate', 'state'],inplace=True)
cont_nn, cat_nn = cont_cat_split(df_nn, 50, dep_var='Repub%of_two-party_vote') # splits into cont or cat based on whether var has more than whatever the specified number (e.g., 50) of different values

In [8]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df_nn, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names='Repub%of_two-party_vote')

In [17]:
dls0 = to_nn.dataloaders(32) # batch size 32, determined from prior experimentation to perform well
xs,y = to_nn.train.xs,to_nn.train.y
valid_xs,valid_y = to_nn.valid.xs,to_nn.valid.y

In [27]:
neural_nets

{0.139041: <fastai.tabular.learner.TabularLearner at 0x1be9e1457f0>,
 0.106517: <fastai.tabular.learner.TabularLearner at 0x1be9deaba30>,
 0.106694: <fastai.tabular.learner.TabularLearner at 0x1be9e065190>,
 0.090117: <fastai.tabular.learner.TabularLearner at 0x1be9dfde730>,
 0.098844: <fastai.tabular.learner.TabularLearner at 0x1be99972f70>,
 0.076316: <fastai.tabular.learner.TabularLearner at 0x1be999f9ac0>,
 0.124174: <fastai.tabular.learner.TabularLearner at 0x1be9dc814c0>,
 0.109538: <fastai.tabular.learner.TabularLearner at 0x1be9df37730>,
 0.086637: <fastai.tabular.learner.TabularLearner at 0x1be9df22c70>,
 0.079053: <fastai.tabular.learner.TabularLearner at 0x1be9df377c0>,
 0.122209: <fastai.tabular.learner.TabularLearner at 0x1be9df4ff70>,
 0.090337: <fastai.tabular.learner.TabularLearner at 0x1be9deb0070>,
 0.11636: <fastai.tabular.learner.TabularLearner at 0x1be9df1bbe0>,
 0.111086: <fastai.tabular.learner.TabularLearner at 0x1be9df8d580>,
 0.089: <fastai.tabular.learner.Tab

In [30]:
# results from some non-nested and thus non-exhaustive loops for hyperparameter tuning, not shown in this notebook,
# but that gives the best performance I've been able to achieve so far:

best_net = load_pickle('0.076_error_neural_net.pkl')

In [31]:
# it appears that the best combination of hyperparameters is batch size 32, ReLU activation, learning rate = 0.001,
# 5 hidden layers of 100 nodes each, trained for 40 epochs

vars(best_net)

{'dls': <fastai.tabular.data.TabularDataLoaders at 0x2a9ae8c6cd0>,
 'model': TabularModel(
   (embeds): ModuleList(
     (0): Embedding(51, 14)
     (1): Embedding(3, 3)
     (2): Embedding(3, 3)
   )
   (emb_drop): Dropout(p=0.0, inplace=False)
   (bn_cont): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (layers): Sequential(
     (0): LinBnDrop(
       (0): Linear(in_features=26, out_features=100, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): LinBnDrop(
       (0): Linear(in_features=100, out_features=100, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (2): LinBnDrop(
       (0): Linear(in_features=100, out_features=100, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [32]:
preds,targs = best_net.get_preds()

# validation set error
r_mse(preds,targs)

0.076316

In [26]:
# Ideally, one would run an (at least moreso) exhaustive hyperparam tuning loop, as below. 
# However, this is time intensive, and I am satisfied with the performance of best_net

neural_nets = {}

nums_of_layers = range(2,21)
nodes_per_layer = [100*x for x in range(1,6)]
learn_rates = (0.0001, 0.0005, 0.001, 0.005, 0.01)
num_epochs = [10*x for x in range(1,31)]
activ_funcs = (nn.ReLU(), nn.Sigmoid())
batch_sizes = (1, 2, 4, 8, 16, 25, 32, 50, 75, 100, 125, 150, 184)

for num_layers in nums_of_layers:
    for num_nodes in nodes_per_layer:
        for rate in learn_rates:
            for epochs in num_epochs:
                for act in activ_funcs():
                    for batch_size in batch_sizes:
                        dls = to_nn.dataloaders(batch_size)
                        learn = tabular_learner(dls, y_range=(0,1), layers=[num_nodes for layer in range(num_layers+1)], config={"act_cls": nn.ReLU()}, 
                                        n_out=1, loss_func=F.mse_loss)
                        learn.fit(epochs, rate)
                        preds,targs = learn.get_preds()
                        error = r_mse(preds, targs)
                        neural_nets[error] = f'{num_layers} layers, {num_nodes} nodes, {rate} learning rate, {epochs} epochs, {act} activations'

epoch,train_loss,valid_loss,time
0,0.045359,0.016019,00:00
1,0.039665,0.015703,00:00
2,0.035733,0.015784,00:00
3,0.032584,0.016211,00:00
4,0.029365,0.016517,00:00
5,0.026659,0.016571,00:00
6,0.024213,0.015982,00:00
7,0.022368,0.015439,00:00
8,0.020796,0.015546,00:00
9,0.019265,0.015328,00:00


epoch,train_loss,valid_loss,time
0,0.024845,0.013893,00:00
1,0.018554,0.013043,00:00
2,0.015431,0.012377,00:00
3,0.013863,0.012004,00:00
4,0.012070,0.011593,00:00
5,0.011175,0.011447,00:00
6,0.010418,0.010780,00:00
7,0.009819,0.010685,00:00
8,0.009335,0.009935,00:00
9,0.008825,0.009282,00:00


epoch,train_loss,valid_loss,time
0,0.027068,0.014252,00:00
1,0.021332,0.014042,00:00
2,0.017282,0.013333,00:00
3,0.014942,0.011239,00:00
4,0.013274,0.010491,00:00
5,0.011943,0.009276,00:00
6,0.010835,0.009273,00:00
7,0.009951,0.008075,00:00
8,0.009184,0.007848,00:00
9,0.008784,0.007166,00:00


epoch,train_loss,valid_loss,time
0,0.030293,0.013552,00:00
1,0.022201,0.015758,00:00
2,0.017934,0.014451,00:00
3,0.015547,0.013163,00:00
4,0.013322,0.011450,00:00
5,0.011838,0.013344,00:00
6,0.010862,0.009803,00:00
7,0.009993,0.010629,00:00
8,0.009092,0.010800,00:00
9,0.008293,0.008732,00:00


epoch,train_loss,valid_loss,time
0,0.032235,0.016059,00:00
1,0.023335,0.008508,00:00
2,0.019182,0.013173,00:00
3,0.016299,0.006823,00:00
4,0.014736,0.007578,00:00
5,0.013161,0.006509,00:00
6,0.011708,0.006353,00:00
7,0.010579,0.006268,00:00
8,0.009524,0.008540,00:00
9,0.008808,0.008582,00:00


...

epoch,train_loss,valid_loss,time
0,0.036594,0.016067,00:00
1,0.027984,0.015178,00:00
2,0.021905,0.014126,00:00
3,0.018594,0.009568,00:00
4,0.015961,0.008898,00:00
5,0.014096,0.007425,00:00
6,0.012893,0.006987,00:00
7,0.011711,0.008299,00:00
8,0.010891,0.007240,00:00
9,0.010185,0.009137,00:00


epoch,train_loss,valid_loss,time
0,0.043034,0.015855,00:00
1,0.030387,0.015594,00:00
2,0.023288,0.009219,00:00
3,0.019246,0.006258,00:00
4,0.016526,0.007223,00:00
5,0.014857,0.008011,00:00
6,0.013684,0.006991,00:00
7,0.012711,0.008760,00:00
8,0.011856,0.009627,00:00
9,0.011080,0.011997,00:00


epoch,train_loss,valid_loss,time
0,0.030487,0.015211,00:00
1,0.023602,0.015633,00:00
2,0.019904,0.016040,00:00
3,0.017971,0.015894,00:00
4,0.016026,0.014619,00:00
5,0.014818,0.011569,00:00
6,0.014071,0.009097,00:00
7,0.012873,0.007836,00:00
8,0.012131,0.008580,00:00
9,0.011552,0.009335,00:00


epoch,train_loss,valid_loss,time
0,0.023677,0.014439,00:00
1,0.020192,0.014492,00:00
2,0.018703,0.014434,00:00
3,0.016560,0.013809,00:00
4,0.015287,0.013147,00:00
5,0.014150,0.012751,00:00
6,0.013327,0.012316,00:00
7,0.012417,0.011817,00:00
8,0.011759,0.011613,00:00
9,0.011149,0.010537,00:00


KeyboardInterrupt: 